In [20]:
from reactionmodel.model import Model, Species, Reaction
import numpy as np
from scipy.special import binom
from numba import jit

In [7]:
x = Species('X')
dimer = Species('2X')
r = Reaction('dimer', [(x, 2)], dimer, k=10.0)
m = Model([x], [r])

In [81]:
y = np.array([9.0, 8.5, 5.0, 10000.0])
k = np.array([15.0, 0.001, 1.0, 2.0])
kinetic_order = np.array([
    [2.0, 2.0, 3.0, 2.0],
    [2.0, 2.0, 3.0, 2.0],
    [2.0, 2.0, 3.0, 2.0],
    [2.0, 2.0, 3.0, 2.0]
]).T

In [70]:
print(np.prod(np.expand_dims(y, axis=1)**kinetic_order, axis=0) * k)

[1.09729688e+15 7.31531250e+10 7.31531250e+13 1.46306250e+14]


In [71]:
%%timeit

np.prod(np.expand_dims(y, axis=1)**kinetic_order, axis=0) * k

4.01 µs ± 58.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [72]:
%%timeit

np.prod(binom(np.expand_dims(y, axis=1), kinetic_order), axis=0) * k

3.93 µs ± 19.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [73]:
print(np.prod(binom(np.expand_dims(y, axis=1), kinetic_order), axis=0) * k)

[8.60538938e+12 5.73692625e+08 5.73692625e+11 1.14738525e+12]


In [80]:
print(binom(np.expand_dims(y, axis=1), kinetic_order))

[[3.6000e+01 3.6000e+01 3.6000e+01 3.6000e+01]
 [3.1875e+01 3.1875e+01 3.1875e+01 3.1875e+01]
 [1.0000e+01 1.0000e+01 1.0000e+01 1.0000e+01]
 [4.9995e+07 4.9995e+07 4.9995e+07 4.9995e+07]]


In [25]:
@jit(nopython=True)
def jit_calculate_propensities(t, y):
    # product along column in kinetic order matrix
    # binom(state, n involvement) (macroscopically ~= #X**n involvement)
    # multiplied by rate constants == propensity
    # dimension of y is expanded to make it a column vector
    #intensity_power = binom(np.expand_dims(y, axis=1), kinetic_order)
    intensity_power = np.expand_dims(y, axis=1)**kinetic_order
    product_down_columns = np.ones(len(k))
    for i in range(0, len(y)):
        product_down_columns = product_down_columns * intensity_power[i]
    return product_down_columns * k

In [27]:
%%timeit

jit_calculate_propensities(0, y)

730 ns ± 6.91 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [33]:
print(jit_calculate_propensities(0, y))

[1.9440e+14 1.2960e+10 4.6656e+19 2.5920e+13]


In [88]:
@jit(nopython=True)
def binom_jit_calculate_propensities(t, y):
    # product along column in kinetic order matrix
    # binom(state, n involvement) (macroscopically ~= #X**n involvement)
    # multiplied by rate constants == propensity
    # dimension of y is expanded to make it a column vector
    intensity_power = np.zeros_like(kinetic_order)
    for i in range(0, kinetic_order.shape[0]):
        for j in range(0, kinetic_order.shape[1]):
            if y[i] < kinetic_order[i][j]:
                intensity_power[i][j] = 0.0
            elif y[i] == kinetic_order[i][j]:
                intensity_power[i][j] = 1.0
            else:
                intensity = 1.0
                for x in range(0, kinetic_order[i][j]):
                    intensity *= (y[i] - x) / (x+1)
                intensity_power[i][j] = intensity

    product_down_columns = np.ones(len(k))
    for i in range(0, len(y)):
        product_down_columns = product_down_columns * intensity_power[i]
    return product_down_columns * k

In [89]:
binom_jit_calculate_propensities(0, y)

array([8.60538938e+12, 5.73692625e+08, 5.73692625e+11, 1.14738525e+12])

In [90]:
%%timeit

binom_jit_calculate_propensities(0, y)

746 ns ± 1.55 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
